In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

        full_df = pd.read_csv(os.path.join(dirname, filename))
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

A look at the columns of the data set:

In [2]:
full_df.head()

In [3]:
print(full_df.info())

As described in the dataset,

> This is data collated from the Bureau of Labor Statistics and Census websites, about the labor force levels and median weekly earnings in constant 2020 dollars, for persons aged 25 and higher, from 1960 onwards.
> 
> Educational attainment levels are categorized as follows: 
>
> LHS: Less than a High School Diploma, 
> 
> HSG: High School Graduation or passing GED or equivalent, 
> 
> CAD: Some College or Associates Degree, 
> 
> BHD: Bachelors or Higher Degree.
> 
> The median weekly earnings data by educational attainment is available from 1979 onwards, while the labor force data by educational attainment is available from 1992 onwards. However, the overall labor force and median weekly earnings data, and the percentage of population with high school graduation and with a bachelors degree or higher is available from 1960 onwards.

The rows for labor force data are unavailable for years 1960-1991, while the median weekly earnings data are unavailable for years 1960-1978, as stated above. However, overall labor force data and overall median weekly earnings (for ages 15 and above) are available.

In [4]:
truncated_df = full_df.dropna()
truncated_df.head()

In [5]:
%matplotlib inline  

# some imports to set up plotting
import matplotlib.pyplot as plt
# pip install seaborn
import seaborn as sns

# Graphics in retina format are more sharp and legible
%config InlineBackend.figure_format = 'retina'

In [6]:
color_dict = {'LHS': 'peru', 'HSG': 'darkorange', 'CAD': 'olivedrab', 'BHD': 'teal', 'overall': 'grey'}
### plotting Median Weekly Earnings ###
# plt.plot(full_df['YEAR'], full_df['OVERALL WEEKLY MEDIAN EARNINGS (AGES 15 AND ABOVE)'],
#          marker='o',linestyle='None', color=color_dict['overall'], label='overall')
plt.plot(full_df['YEAR'], full_df['LHS MEDIAN WEEKLY EARNING (IN 2020 DOLLARS)'],
         marker='o',linestyle='None', color=color_dict['LHS'], label='LHS')
plt.hlines(full_df['LHS MEDIAN WEEKLY EARNING (IN 2020 DOLLARS)'].to_numpy()[19],
           full_df['YEAR'].to_numpy()[19], full_df['YEAR'].to_numpy()[-1], linestyle='dashed', 
           color=color_dict['LHS'])
plt.plot(full_df['YEAR'], full_df['HSG MEDIAN WEEKLY EARNING (IN 2020 DOLLARS)'],
         marker='o',linestyle='None', color=color_dict['HSG'], label='HSG')
plt.hlines(full_df['HSG MEDIAN WEEKLY EARNING (IN 2020 DOLLARS)'].to_numpy()[19],
           full_df['YEAR'].to_numpy()[19], full_df['YEAR'].to_numpy()[-1], linestyle='dashed', 
           color=color_dict['HSG'])
plt.plot(full_df['YEAR'], full_df['CAD MEDIAN WEEKLY EARNING (IN 2020 DOLLARS)'],
         marker='o',linestyle='None', color=color_dict['CAD'], label='CAD')
plt.hlines(full_df['CAD MEDIAN WEEKLY EARNING (IN 2020 DOLLARS)'].to_numpy()[19],
           full_df['YEAR'].to_numpy()[19], full_df['YEAR'].to_numpy()[-1], linestyle='dashed', 
           color=color_dict['CAD'])

plt.plot(full_df['YEAR'], full_df['BHD MEDIAN WEEKLY EARNING (IN 2020 DOLLARS)'],
         marker='o',linestyle='None', color=color_dict['BHD'], label='BHD')
plt.hlines(full_df['BHD MEDIAN WEEKLY EARNING (IN 2020 DOLLARS)'].to_numpy()[19],
           full_df['YEAR'].to_numpy()[19], full_df['YEAR'].to_numpy()[-1], linestyle='dashed', 
           color=color_dict['BHD'])

plt.plot(full_df['YEAR'], full_df['OVERALL WEEKLY MEDIAN EARNINGS (AGES 15 AND ABOVE)'],
         marker='o',linestyle='None', color=color_dict['overall'], label='overall')

plt.legend()
plt.ylabel('Median Weekly Earnings (in 2020 dollars)')
plt.xlabel('Year')
plt.show()

plt.plot(full_df['YEAR'], full_df['OVERALL PERCENT HSG'],          
         marker='o',linestyle='None', color=color_dict['overall'], label='overall HSG')
plt.plot(full_df['YEAR'], full_df['OVERALL PERCENT BHD'],          
         marker='o',linestyle='None', color='c', label='overall BHD')
plt.legend()
plt.ylabel('Population Level Percentages')
plt.xlabel('Year')
plt.show()

The above plots provide the motivation for the dataset and this notebook.
We notice that for almost all educational attainment categories except BHD (Bachelors or Higher Degree), the median weekly income has stayed stagnant or decreased since 1979.

The second plot shows the percentage of population that are High School Graduates (HSG), and the percentage of population with a Bachelors or Higher Degree (BHD). Notice the large gap between the HSG percentage and BHD percentage which has widened over the years.

Given the low rates of college graduation in the country, the stagnation/decrease of median weekly incomes for people without a Bachelors or Higher Degree is an alarming trend, suggesting that majority of the country's labor force has seen a stagnation or decrease in their weekly incomes since the 1980s.

We want to predict the future trend for the labor force statistics and median weekly incomes by educational attainment categories.

Our goals are: 
(i) to estimate the labor force data for the future years 2020 to 2030 from past labor force data
(ii) to estimate the median weekly earnings by highest educational attainment for the future years 2020 to 2030 from past earnings data and past and predicted labor force data.

We note from the median weekly earnings plot above, that there is a "seasonal" trend that aligns with the periodically occuring financial crises, in addition to a local linear trend for each of the categories.

We will begin with an exploratory data analysis of this dataset.

In [7]:
### plotting Population Percentages & Overall Percentage of HSG & BHD ###
plt.plot(truncated_df['YEAR'], 
      100*(truncated_df['LHS CIVILIAN POPULATION (IN THOUSANDS)']/truncated_df['CIVILIAN NONINSTITUTIONAL POPULATION (IN THOUSANDS)']),
         marker='o',linestyle='None', color=color_dict['LHS'], label='LHS')
plt.plot(truncated_df['YEAR'],
      100*(truncated_df['HSG CIVILIAN POPULATION (IN THOUSANDS)']/truncated_df['CIVILIAN NONINSTITUTIONAL POPULATION (IN THOUSANDS)']),
         marker='o',linestyle='None', color=color_dict['HSG'], label='HSG')
plt.plot(truncated_df['YEAR'], 100*(truncated_df['CAD CIVILIAN POPULATION (IN THOUSANDS)']/truncated_df['CIVILIAN NONINSTITUTIONAL POPULATION (IN THOUSANDS)']),
         marker='o',linestyle='None', color=color_dict['CAD'], label='CAD')
plt.plot(truncated_df['YEAR'], 
      100*(truncated_df['BHD CIVILIAN POPULATION (IN THOUSANDS)']/truncated_df['CIVILIAN NONINSTITUTIONAL POPULATION (IN THOUSANDS)']),
         marker='o',linestyle='None', color=color_dict['BHD'], label='BHD')
plt.plot(full_df['YEAR'], full_df['OVERALL PERCENT HSG'],          
         marker='o',linestyle='None', color=color_dict['overall'], label='overall HSG')
plt.plot(full_df['YEAR'], full_df['OVERALL PERCENT BHD'],          
         marker='o',linestyle='None', color='c', label='overall BHD')
plt.legend()
plt.ylabel('Population Level Fractions')
plt.xlabel('Year')
plt.show()

Here we see that the percentage of BHD (black) has the same behavior as the population percent of BHD (teal), and (100 - the percentage of HSG) (cyan) has the same behavior as population fraction of LHS (peru), as we might expect. We also find that the population fraction of CAD (olive) and that of HSG (orange) behaves similar to linear combinations of the overall percentage of BHD and the overall percentage of HSG. We can perform a regression analysis to determine the best estimates of the CAD population fraction and the HSG population fraction from the overall percent of BHD and HSG. 

In [8]:
### plotting Labor Force Levels ###
plt.plot(truncated_df['YEAR'], truncated_df['LHS LABOR FORCE LEVEL (IN THOUSANDS)'],
         marker='o',linestyle='None', color=color_dict['LHS'], label='LHS')
plt.plot(truncated_df['YEAR'], truncated_df['HSG LABOR FORCE LEVEL (IN THOUSANDS)'],
         marker='o',linestyle='None', color=color_dict['HSG'], label='HSG')
plt.plot(truncated_df['YEAR'], truncated_df['CAD LABOR FORCE LEVEL (IN THOUSANDS)'],
         marker='o',linestyle='None', color=color_dict['CAD'], label='CAD')
plt.plot(truncated_df['YEAR'], truncated_df['BHD LABOR FORCE LEVEL (IN THOUSANDS)'],
         marker='o',linestyle='None', color=color_dict['BHD'], label='BHD')
plt.plot(full_df['YEAR'], full_df['OVERALL LABOR FORCE LEVEL (IN THOUSANDS)']/4.,
         marker='o',linestyle='None', color=color_dict['overall'], label='overall/4')
plt.legend()
plt.ylabel('Labor Force Level (in Thousands)')
plt.xlabel('Year')
plt.show()

### plotting Labor Force Participation Rates ###
plt.plot(truncated_df['YEAR'], truncated_df['LHS LABOR FORCE PARTICIPATION RATE (IN PERCENT)'],
         marker='o',linestyle='None', color=color_dict['LHS'], label='LHS')
plt.plot(truncated_df['YEAR'], truncated_df['HSG LABOR FORCE PARTICIPATION RATE (IN PERCENT)'],
         marker='o',linestyle='None', color=color_dict['HSG'], label='HSG')
plt.plot(truncated_df['YEAR'], truncated_df['CAD LABOR FORCE PARTICIPATION RATE (IN PERCENT)'],
         marker='o',linestyle='None', color=color_dict['CAD'], label='CAD')
plt.plot(truncated_df['YEAR'], truncated_df['BHD LABOR FORCE PARTICIPATION RATE (IN PERCENT)'],
         marker='o',linestyle='None', color=color_dict['BHD'], label='BHD')
plt.plot(full_df['YEAR'], full_df['OVERALL LABOR FORCE PARTICIPATION RATE (IN PERCENT)'],
         marker='o',linestyle='None', color=color_dict['overall'], label='overall')
plt.legend()
plt.xlabel('Year')
plt.ylabel('Labor Force Participation Rate (in Percent)')
plt.show()

### plotting Employment Levels ###
plt.plot(truncated_df['YEAR'], truncated_df['LHS EMPLOYMENT LEVEL (IN THOUSANDS)'],
         marker='o',linestyle='None', color=color_dict['LHS'], label='LHS')
plt.plot(truncated_df['YEAR'], truncated_df['HSG EMPLOYMENT LEVEL (IN THOUSANDS)'],
         marker='o',linestyle='None', color=color_dict['HSG'], label='HSG')
plt.plot(truncated_df['YEAR'], truncated_df['CAD EMPLOYMENT LEVEL (IN THOUSANDS)'],
         marker='o',linestyle='None', color=color_dict['CAD'], label='CAD')
plt.plot(truncated_df['YEAR'], truncated_df['BHD EMPLOYMENT LEVEL (IN THOUSANDS)'],
         marker='o',linestyle='None', color=color_dict['BHD'], label='BHD')
plt.plot(full_df['YEAR'], full_df['OVERALL EMPLOYMENT LEVEL (IN THOUSANDS)']/4.,
         marker='o',linestyle='None', color=color_dict['overall'], label='overall/4')
plt.legend()
plt.ylabel('Employment Level (in Thousands)')
plt.xlabel('Year')
plt.show()

It looks like for each of plots above, the overall quantity behaves like the sum/average of the corresponding quantity for LHS, HSG, CAD and BHD categories, as it should.

Furthermore, we don't see the seasonal trend that was present in the median weekly incomes in any of the above labor force statistics, except perhaps in the employment level.

We would like to predict for the future years, for each of categories (LHS, HSG, CAD, BHD), the independent Labor force variables: Labor Force Level, Labor Force Participation Rate, and the Labor Force Employment Level. 

Let us begin by calculating the autocorrelation for the Labor Force levels. This will tell us how predictive past values are for future values, and also how many years into the past are relevant for the current level.

In [9]:
import statsmodels.api as sm                    
sm.graphics.tsa.plot_acf(full_df['OVERALL LABOR FORCE LEVEL (IN THOUSANDS)'], lags=10, 
                 alpha=0.05, title='Overall Labor Force Level Autocorrelation')
plt.xlabel('Lag')
plt.show()

sm.graphics.tsa.plot_acf(truncated_df['LHS LABOR FORCE LEVEL (IN THOUSANDS)'], lags=10, alpha=0.05, 
                         title='LHS Labor Force Level Autocorrelation')
plt.xlabel('Lag')
plt.show()

sm.graphics.tsa.plot_acf(truncated_df['HSG LABOR FORCE LEVEL (IN THOUSANDS)'], lags=10, alpha=0.05, 
                         title='HSG Labor Force Level Autocorrelation' )
plt.xlabel('Lag')
plt.show()

sm.graphics.tsa.plot_acf(truncated_df['CAD LABOR FORCE LEVEL (IN THOUSANDS)'], lags=10, alpha=0.05, 
                         title='CAD Labor Force Level Autocorrelation' )
plt.xlabel('Lag')
plt.show()

sm.graphics.tsa.plot_acf(truncated_df['BHD LABOR FORCE LEVEL (IN THOUSANDS)'], lags=10, alpha=0.05, 
                         title='BHD Labor Force Level Autocorrelation' )
plt.xlabel('Lag')
plt.show()

We see that while the overall labor force level shows history dependence for upto 5 years, the labor force levels for each category show history dependence for only upto 2 years.

We will now plot the autocorrelation for the other labor force statistics. 

In [10]:
sm.graphics.tsa.plot_acf(full_df['OVERALL LABOR FORCE PARTICIPATION RATE (IN PERCENT)'], lags=10, alpha=0.05, 
                         title='Overall Labor Force Participation Rate Autocorrelation' )
plt.xlabel('Lag')
plt.show()

sm.graphics.tsa.plot_acf(truncated_df['LHS LABOR FORCE PARTICIPATION RATE (IN PERCENT)'], lags=10, alpha=0.05, 
                         title='LHS LF Participation Rate Autocorrelation' )
plt.xlabel('Lag')
plt.show()

sm.graphics.tsa.plot_acf(truncated_df['HSG LABOR FORCE PARTICIPATION RATE (IN PERCENT)'], lags=10, alpha=0.05, 
                         title='HSG LF Participation Rate Autocorrelation' )
plt.xlabel('Lag')
plt.show()

sm.graphics.tsa.plot_acf(truncated_df['CAD LABOR FORCE PARTICIPATION RATE (IN PERCENT)'], lags=10, alpha=0.05, 
                         title='CAD LF Participation Rate Autocorrelation' )
plt.xlabel('Lag')
plt.show()

sm.graphics.tsa.plot_acf(truncated_df['BHD LABOR FORCE PARTICIPATION RATE (IN PERCENT)'], lags=10, alpha=0.05, 
                         title='BHD LF Participation Rate Autocorrelation' )
plt.xlabel('Lag')
plt.show()

The autocorrelation for the Labor Force Participation Rate also suggests a maximum history dependence for upto 2 years for the category-wise data. 

We will now look at the autocorrelation for the Employment Level.

In [11]:
sm.graphics.tsa.plot_acf(full_df['OVERALL EMPLOYMENT LEVEL (IN THOUSANDS)'], lags=10, 
                 alpha=0.05, title='Overall Employment Level Autocorrelation')
plt.xlabel('Lag')
plt.show()

sm.graphics.tsa.plot_acf(truncated_df['LHS EMPLOYMENT LEVEL (IN THOUSANDS)'], lags=10, alpha=0.05, 
                         title='LHS Employment Level Autocorrelation')
plt.xlabel('Lag')
plt.show()

sm.graphics.tsa.plot_acf(truncated_df['HSG EMPLOYMENT LEVEL (IN THOUSANDS)'], lags=10, alpha=0.05, 
                         title='HSG Employment Level Autocorrelation' )
plt.xlabel('Lag')
plt.show()

sm.graphics.tsa.plot_acf(truncated_df['CAD EMPLOYMENT LEVEL (IN THOUSANDS)'], lags=10, alpha=0.05, 
                         title='CAD Employment Level Autocorrelation' )
plt.xlabel('Lag')
plt.show()

sm.graphics.tsa.plot_acf(truncated_df['BHD EMPLOYMENT LEVEL (IN THOUSANDS)'], lags=10, alpha=0.05, 
                         title='BHD Employment Level Autocorrelation' )
plt.xlabel('Lag')
plt.show()

The autocorrelation for the Employment also suggests a maximum history dependence for upto 2 years for the category-wise data.

Since we want to forecast values of the labor force variables and the median earnings for a decade into the future and not just one year into the future, we  will use a Bayesian Structural Time Series model from the Tensor Flow Probability package instead of other machine learning methods which require much more data to fit, and often involve errors that accumulate for future predictions. We will use [this fantastic example notebook from TensorFlow Probability](https://colab.research.google.com/github/tensorflow/probability/blob/main/tensorflow_probability/examples/jupyter_notebooks/Structural_Time_Series_Modeling_Case_Studies_Atmospheric_CO2_and_Electricity_Demand.ipynb#scrollTo=ULm_Z8Oe0Lhd) as our guide.

The plots above suggest that the there is a local linear trend with a small seasonal component. 
We now embark on the estimation of the past labor force statistics, beginning with importing the necessary packages and writing some plotting functions.

In [12]:
import collections

import tensorflow.compat.v2 as tf
import tensorflow_probability as tfp

from tensorflow_probability import distributions as tfd
from tensorflow_probability import sts

tf.enable_v2_behavior()

def plot_forecast(x, y,
                  forecast_mean, forecast_scale, forecast_samples,
                  title):
    colors = sns.color_palette()
    c1, c2 = colors[0], colors[1]
    fig = plt.figure(figsize=(12, 6))
    ax = fig.add_subplot(1, 1, 1)

    num_steps = len(y)
    num_steps_forecast = forecast_mean.shape[-1]
    num_steps_train = num_steps - num_steps_forecast
        
    ax.plot(x, y, lw=2, color=c1, label='ground truth')

    forecast_steps = np.arange(
      x[num_steps_train],
      x[num_steps_train]+num_steps_forecast,
      dtype=x.dtype)
    
    mape = 100.*np.mean(np.abs(forecast_mean - 
                         y[-num_steps_forecast:])/y[-num_steps_forecast:])
    print(f"MAPE is {mape:.2f}%")
    ax.plot(forecast_steps, (forecast_samples.T), lw=1, color=c2, alpha=0.4)
    
    ax.plot(forecast_steps, forecast_mean, lw=2, ls='--', color=c2,
           label='forecast')
    ax.fill_between(forecast_steps,
                   forecast_mean-2*forecast_scale,
                   forecast_mean+2*forecast_scale, color=c2, alpha=0.2)

    ymin, ymax = min(np.min(forecast_samples), np.min(y)), max(np.max(forecast_samples), np.max(y))
    yrange = ymax-ymin
    ax.set_ylim([ymin - yrange*0.1, ymax + yrange*0.1])
    ax.set_title("{}".format(title))
    ax.legend()

    return fig, ax

def plot_components(dates,
                    component_means_dict,
                    component_stddevs_dict):
    colors = sns.color_palette()
    c1, c2 = colors[0], colors[1]

    axes_dict = collections.OrderedDict()
    num_components = len(component_means_dict)
    fig = plt.figure(figsize=(12, 2.5 * num_components))
    for i, component_name in enumerate(component_means_dict.keys()):
        component_mean = component_means_dict[component_name]
        component_stddev = component_stddevs_dict[component_name]

    ax = fig.add_subplot(num_components,1,1+i)
    ax.plot(dates, component_mean, lw=2)
    ax.fill_between(dates,
                     component_mean-2*component_stddev,
                     component_mean+2*component_stddev,
                     color=c2, alpha=0.5)
    ax.set_title(component_name)
    axes_dict[component_name] = ax
    fig.tight_layout()
    return fig, axes_dict

For each category and the labor force variables, we will now build a BSTS model that has a seasonal component and a semi-local linear trend component, to estimate the values for the years 2021-2030.

Moreover, since we have more data for the overall labor force variables (1960 onwards) which also have a higher autocorrelation, and since we have the population fractions for persons with a High School Graduation and persons with a Bachelors Degree or Higher, we will forecast values of these variables first, and use these variables to forecast the future values of the category-wise labor force variables. We begin by collating the overall and category-wise data, and writing the model-building functions.

We will use a semi-local (to keep the variance from getting too large) linear trend model, and a combination of the semi-local linear trend with a seasonal trend component for modeling the roughly periodically occurring financial crises.

In [13]:
overall_hsg_percent = full_df['OVERALL PERCENT HSG'][4:].to_numpy()
overall_bhd_percent = full_df['OVERALL PERCENT BHD'][4:].to_numpy()

### [4:] since elements 1 and 3 are missing. ###
### We will use [4:] for all the overall labor force variables ### 

overall_lf_level = 1.*full_df['OVERALL LABOR FORCE LEVEL (IN THOUSANDS)'][4:].to_numpy()
lhs_lf_level = 1.*truncated_df['LHS LABOR FORCE LEVEL (IN THOUSANDS)'].to_numpy()
hsg_lf_level = 1.*truncated_df['HSG LABOR FORCE LEVEL (IN THOUSANDS)'].to_numpy()
cad_lf_level = 1.*truncated_df['CAD LABOR FORCE LEVEL (IN THOUSANDS)'].to_numpy()
bhd_lf_level = 1.*truncated_df['BHD LABOR FORCE LEVEL (IN THOUSANDS)'].to_numpy()

overall_emp_level = 1.*full_df['OVERALL EMPLOYMENT LEVEL (IN THOUSANDS)'][4:].to_numpy()
lhs_emp_level = 1.*truncated_df['LHS EMPLOYMENT LEVEL (IN THOUSANDS)'].to_numpy()
hsg_emp_level = 1.*truncated_df['HSG EMPLOYMENT LEVEL (IN THOUSANDS)'].to_numpy()
cad_emp_level = 1.*truncated_df['CAD EMPLOYMENT LEVEL (IN THOUSANDS)'].to_numpy()
bhd_emp_level = 1.*truncated_df['BHD EMPLOYMENT LEVEL (IN THOUSANDS)'].to_numpy()

lhs_lf_pr = 1.*truncated_df['LHS LABOR FORCE PARTICIPATION RATE (IN PERCENT)'].to_numpy()
hsg_lf_pr = 1.*truncated_df['HSG LABOR FORCE PARTICIPATION RATE (IN PERCENT)'].to_numpy()
cad_lf_pr = 1.*truncated_df['CAD LABOR FORCE PARTICIPATION RATE (IN PERCENT)'].to_numpy()
bhd_lf_pr = 1.*truncated_df['BHD LABOR FORCE PARTICIPATION RATE (IN PERCENT)'].to_numpy()
overall_lf_pr = 1.*full_df['OVERALL LABOR FORCE PARTICIPATION RATE (IN PERCENT)'][4:].to_numpy()

full_df_years = 1.*full_df['YEAR'][4:].to_numpy()
trunc_df_years = 1.*truncated_df['YEAR'].to_numpy()
num_forecast_steps = 10 ### since we want to forecast 10 years into the future ###

def build_linear_trend_seasonal_model(observed_time_series, steps_per_season=1):
    semilocal_linear_trend = sts.SemiLocalLinearTrend(
                                observed_time_series=observed_time_series,
                                name='semilocal_linear_trend')    

    seasonal_trend = sts.Seasonal(
            num_seasons=2, num_steps_per_season=steps_per_season, 
            observed_time_series=observed_time_series)

    model = sts.Sum([semilocal_linear_trend, seasonal_trend],
                   observed_time_series=observed_time_series)

    return model

def build_linear_trend_model(observed_time_series):
    semilocal_linear_trend = sts.SemiLocalLinearTrend(
                                observed_time_series=observed_time_series,
                                name='semilocal_linear_trend')    

    model = sts.Sum([semilocal_linear_trend],
                   observed_time_series=observed_time_series)

    return model

Functions for fitting the model and generating the forecast:

In [14]:
def fit_model_generate_forecast(model, train_time_series, num_forecast_steps, 
                                num_variational_steps=200):
# Build the variational surrogate posteriors `qs`.
    np.random.seed(101)
    high = np.iinfo(np.int32).max
    seed = np.random.randint(high)
    variational_posteriors = tfp.sts.build_factored_surrogate_posterior(model=model, 
                                                                        seed=seed)

# Build and optimize the variational loss function.
    elbo_loss_curve = tfp.vi.fit_surrogate_posterior(
        target_log_prob_fn=model.joint_log_prob(
            observed_time_series=train_time_series),
        surrogate_posterior=variational_posteriors,
        optimizer=tf.optimizers.Adam(learning_rate=0.1),
        num_steps=num_variational_steps,
        jit_compile=True, seed=seed)
    plt.plot(elbo_loss_curve)
    plt.show()

    q_samples_ = variational_posteriors.sample(100)
    forecast_dist = tfp.sts.forecast(
        model=model,
        observed_time_series=train_time_series,
        parameter_samples=q_samples_,
        num_steps_forecast=num_forecast_steps)
    
    return (variational_posteriors, forecast_dist)

def print_fit_params(variational_posteriors, model):
        # # Draw samples from the variational posterior.
    q_samples_ = variational_posteriors.sample(100)

    print("Inferred parameters:")
    for param in model.parameters:
      print("{}: {} +- {}".format(param.name,
                                  np.mean(q_samples_[param.name], axis=0),
                                  np.std(q_samples_[param.name], axis=0)))

def plot_forecast_results(forecast_dist, x_axis_data, train_data, title_str, 
                          num_samples=10):
    (
        forecast_mean,
        forecast_scale,
        forecast_samples
    ) = (
        forecast_dist.mean().numpy()[..., 0],
        forecast_dist.stddev().numpy()[..., 0],
        forecast_dist.sample(num_samples).numpy()[..., 0]
        )

    fig, ax = plot_forecast(x_axis_data, train_data,
                        forecast_mean,
                        forecast_scale,
                        forecast_samples,
                        title=title_str)
    
    
    fig.tight_layout()
    plt.show()

Applying the above machinery to forecast values of Overall HSG percent, Overall BHD percent, Overall Labor Force Level, Overall Labor Force Participation Rate, and the Overall Employment Level. 

In [15]:
### To estimate the seasonal/oscillating component of the variables ###
from scipy.fft import fft

num_forecast_steps = 10

hsg_percent_fft = fft(overall_hsg_percent[:-num_forecast_steps]-
                 np.mean(overall_hsg_percent[:-num_forecast_steps]))
plt.plot(np.abs(hsg_percent_fft))
plt.grid()
plt.title('Overall HSG Percent FFT')
plt.show()

bhd_percent_fft = fft(overall_bhd_percent[:-num_forecast_steps]-
                 np.mean(overall_bhd_percent[:-num_forecast_steps]))
plt.plot(np.abs(bhd_percent_fft))
plt.grid()
plt.title('Overall BHD Percent FFT')
plt.show()

overall_lf_fft = fft(overall_lf_level[:-num_forecast_steps]-
                np.mean(overall_lf_level[:-num_forecast_steps]))
plt.plot(np.abs(overall_lf_fft))
plt.grid()
plt.title('Overall Labor Force Level FFT')
plt.show()

overall_emp_fft = fft(overall_emp_level[:-num_forecast_steps]-
                np.mean(overall_emp_level[:-num_forecast_steps]))
plt.plot(np.abs(overall_emp_fft))
plt.grid()
plt.title('Overall Employment Level FFT')
plt.show()

overall_lf_pr_fft = fft(overall_lf_pr[:-num_forecast_steps]-
                np.mean(overall_lf_pr[:-num_forecast_steps]))
plt.plot(np.abs(overall_lf_pr_fft))
plt.grid()
plt.title('Overall Labor Force Participation Rate FFT')
plt.show()

The FFT peaks at 1 year, which could could simply be the high frequency noise. We will set the number of steps per season to the shortest possible value: 1, to account for this "periodic" variation.

In [16]:
### Overall HSG percent ###
hsg_percent_model = build_linear_trend_model(overall_hsg_percent[:-num_forecast_steps])
(hsg_percent_var_post, 
 hsg_percent_forecast_dist) = fit_model_generate_forecast(hsg_percent_model,
                                overall_hsg_percent[:-num_forecast_steps], 
                                num_forecast_steps)

# print_fit_params(hsg_percent_var_post, hsg_percent_model)

plot_forecast_results(hsg_percent_forecast_dist, full_df_years, overall_hsg_percent, 
                      'Overall HSG percent', num_samples=10)

It looks like the Semi-local linear trend model does a good job of forecasting the percent of persons with high school graduation over the last decade. 

In [17]:
### Overall BHD percent ###
bhd_percent_model = build_linear_trend_model(overall_bhd_percent[:-num_forecast_steps])
(bhd_percent_var_post, 
 bhd_percent_forecast_dist) = fit_model_generate_forecast( bhd_percent_model,
                overall_bhd_percent[:-num_forecast_steps], num_forecast_steps)

# print_fit_params(bhd_percent_var_post, bhd_percent_model)

plot_forecast_results(bhd_percent_forecast_dist, full_df_years, overall_bhd_percent, 
                      'Overall BHD percent', num_samples=10)

It looks like the percentage of persons with a Bachelors Degree or Higher has increased considerably in the last decade compared to past decades, and the model is unable to forecast that.

So far we have used only a semilocal (to keep the variance bounded) linear trend model, but for the labor force variables, we will also use a seasonal trend in addition to the linear trend, to account for the bumpiness of the data. 

In [18]:
### Overall Labor Force Level ###
### We use a linear trend combined with a seasonal trend here ###
lf_level_model = build_linear_trend_seasonal_model(overall_lf_level[:-num_forecast_steps])
(lf_level_var_post, 
 lf_level_forecast_dist) = fit_model_generate_forecast( lf_level_model,
                        overall_lf_level[:-num_forecast_steps], num_forecast_steps)

# print_fit_params(lf_level_var_post, lf_level_model)

plot_forecast_results(lf_level_forecast_dist, full_df_years, overall_lf_level, 
                      'Overall Labor Force Level', num_samples=10)

It appears that the labor force levels have also shown an uptick that may be correlated with the uptick the BHD percentage of the population, which the forecast is unable to predict. The dip at the end is probably because of Covid, and is again absent in the forecast. Yet, the forecast is only a few percent off from the actual statistics.

In [19]:
### Overall Employment Level ###
### We use a linear trend 
emp_level_model = build_linear_trend_seasonal_model(overall_emp_level[:-num_forecast_steps])
(emp_level_var_post, 
 emp_level_forecast_dist) = fit_model_generate_forecast(emp_level_model,
                        overall_emp_level[:-num_forecast_steps], num_forecast_steps)

# print_fit_params(emp_level_var_post, emp_level_model)

plot_forecast_results(emp_level_forecast_dist, full_df_years, overall_emp_level, 
                      'Overall Employment Level', num_samples=10)

Here again, the forecast is unable to account for initial uptick and the dip likely due to Covid. Yet, it does a reasonably good job, with an error of within a few percent.

In [20]:
### Overall Labor Force Participation Rate ###
lf_pr_model = build_linear_trend_seasonal_model(overall_lf_pr[:-num_forecast_steps])  
(lf_pr_var_post, 
 lf_pr_forecast_dist) = fit_model_generate_forecast( lf_pr_model,
                            overall_lf_pr[:-num_forecast_steps], num_forecast_steps)

# print_fit_params(lf_pr_var_post, lf_pr_model)

plot_forecast_results(lf_pr_forecast_dist, full_df_years, overall_lf_pr, 
                      'Overall Labor Force Participation Rate', num_samples=10)

The late resurgence and eventual dip due to Covid is not captured by the forecast, yet the mean absolute error is within a percent of the actual the Labor Force Participation rate.

We will now use the above models to forecast the Overall Labor Force Variables: HSG Percent, BHD Percent, Labor Force Level, Employment Level, Labor Force Participation Rate, for the years 2021-2030.

In [21]:
#### Functions for generating forecasts from predefined models 
#### and variational posteriors

def generate_forecast(model, var_post, observed_time_series, num_forecast_steps):
    q_samples_ = var_post.sample(100)
    forecast_dist = tfp.sts.forecast(
        model=model,
        observed_time_series=observed_time_series,
        parameter_samples=q_samples_,
        num_steps_forecast=num_forecast_steps)
    forecast_mean = forecast_dist.mean().numpy()[..., 0]
    
    return (forecast_mean, forecast_dist)

def plot_forecast_future(forecast_dist, x, y, title_str, 
                          num_samples=10):
    (
        forecast_mean,
        forecast_scale,
        forecast_samples
    ) = (
        forecast_dist.mean().numpy()[..., 0],
        forecast_dist.stddev().numpy()[..., 0],
        forecast_dist.sample(num_samples).numpy()[..., 0]
        )

    colors = sns.color_palette()
    c1, c2 = colors[0], colors[1]
    fig = plt.figure(figsize=(12, 6))
    ax = fig.add_subplot(1, 1, 1)

    num_steps_train = len(y)
    num_steps_forecast = forecast_mean.shape[-1]

    ax.plot(x[:num_steps_train], y, lw=2, color=c1, label='observed')

    forecast_steps = np.arange(
      x[num_steps_train],
      x[num_steps_train]+num_steps_forecast,
      dtype=x.dtype)
    
    ax.plot(forecast_steps, (forecast_samples.T), lw=1, color=c2, alpha=0.4)
    
    ax.plot(forecast_steps, forecast_mean, lw=2, ls='--', color=c2,
           label='forecast')
    ax.plot(x[num_steps_train-1:num_steps_train+1], np.array([y[-1],forecast_mean[0]]), 
            lw=2, ls='--', color=c1)
    
    ax.fill_between(forecast_steps,
                   forecast_mean-2*forecast_scale,
                   forecast_mean+2*forecast_scale, color=c2, alpha=0.2)

    ymin, ymax = min(np.min(forecast_samples), np.min(y)), max(np.max(forecast_samples), np.max(y))
    yrange = ymax-ymin
    ax.set_ylim([ymin - yrange*0.1, ymax + yrange*0.1])
    ax.set_title("{}".format(title_str))
    ax.legend()
    
    fig.tight_layout()
    plt.show()

We begin with the overall data variables: HSG percent, BHD percent, LF Level, Emp Level, LF Participation Rate.

In [22]:
forecast_years = np.array(range(2021,2031))
num_forecast_steps = 10
hsg_percent_mean, hsg_percent_dist = generate_forecast(hsg_percent_model, 
                                                       hsg_percent_var_post,
                                                       overall_hsg_percent,
                                                       num_forecast_steps)

full_df_forecast_yrs = np.hstack((full_df_years, forecast_years))
plot_forecast_future(hsg_percent_dist, full_df_forecast_yrs, overall_hsg_percent, 
                     'HSG percent')
hsg_percent_forecast = np.hstack((overall_hsg_percent, hsg_percent_mean))

In [23]:
bhd_percent_mean, bhd_percent_dist = generate_forecast(bhd_percent_model, 
                                                       bhd_percent_var_post,
                                                       overall_bhd_percent,
                                                       num_forecast_steps)
plot_forecast_future(bhd_percent_dist, full_df_forecast_yrs, overall_bhd_percent, 
                     'BHD percent')
bhd_percent_forecast = np.hstack((overall_bhd_percent, bhd_percent_mean))

We see that while the model predicts a continuing growth in the population percentage of high school graduates (HSG), it predicts a saturation in the population percentage of persons with a bachelors or higher degree (BHD).

In [24]:
lf_level_mean, lf_level_dist = generate_forecast(lf_level_model, 
                                               lf_level_var_post,
                                               overall_lf_level,
                                               num_forecast_steps)
plot_forecast_future(lf_level_dist, full_df_forecast_yrs, overall_lf_level, 
                     'Overall LF Level')
lf_level_forecast = np.hstack((overall_lf_level, lf_level_mean))

In [25]:
emp_level_mean, emp_level_dist = generate_forecast(emp_level_model, 
                                               emp_level_var_post,
                                               overall_emp_level,
                                               num_forecast_steps)
plot_forecast_future(emp_level_dist, full_df_forecast_yrs, overall_emp_level, 
                     'Overall Employment Level')
emp_level_forecast = np.hstack((overall_emp_level, emp_level_mean))

In [26]:
lf_pr_mean, lf_pr_dist = generate_forecast(lf_pr_model, 
                                               lf_pr_var_post,
                                               overall_lf_pr,
                                               num_forecast_steps)
plot_forecast_future(lf_pr_dist, full_df_forecast_yrs, overall_lf_pr, 
                     'Overall LF Participation Rate')
lf_pr_forecast = np.hstack((overall_lf_pr, lf_pr_mean))

While the HSG percent and BHD percent forecasts show an upward trend, all the Labor Force variables show a downward trend especially because of the impact of Covid in the year 2020.

We are interested in forecasting the educational category-wise labor force variables and the educational category-wise median weekly earnings. For this, we will forecast the category-wise data for the labor force variables in two ways: (i) using only the past category-wise data for that variable, and (ii) using the past category-wise data and the overall data along with the population fractions of HSG and BHD multipied with the overall data for that variable as a regression variable. We will use a semilocal linear trend and a seasonal trend with a seasonal duration determined by calculating a Fourier Transform. 

Finally we will use these forecasts of the category-wise labor force variables and the past data for the category-wise median weekly earnings to forecast the category-wise median weekly earnings.

In [27]:
### To estimate the seasonal/oscillating component of the variable ###
num_forecast_steps = 6

lhs_lf_fft = fft(lhs_lf_level[:-num_forecast_steps]-
                 np.mean(lhs_lf_level[:-num_forecast_steps]))
plt.plot(np.abs(lhs_lf_fft))
plt.grid()
plt.title('LHS LF Level FFT')
plt.show()

hsg_lf_fft = fft(hsg_lf_level[:-num_forecast_steps]-
                np.mean(hsg_lf_level[:-num_forecast_steps]))
plt.plot(np.abs(hsg_lf_fft))
plt.grid()
plt.title('HSG LF Level FFT')
plt.show()

cad_lf_fft = fft(cad_lf_level[:-num_forecast_steps]-
                np.mean(cad_lf_level[:-num_forecast_steps]))
plt.plot(np.abs(cad_lf_fft))
plt.grid()
plt.title('CAD LF Level FFT')
plt.show()

bhd_lf_fft = fft(bhd_lf_level[:-num_forecast_steps]-
                np.mean(bhd_lf_level[:-num_forecast_steps]))
plt.plot(np.abs(bhd_lf_fft))
plt.grid()
plt.title('BHD LF Level FFT')
plt.show()

The above Fourier Transforms show peaks at 5 years for the LHS and HSG, at 4 years for the CAD LF Level, and only the usual peak at 1 year for the BHD LF Level. We will set the number of steps per season accordingly. 

In [45]:
### To estimate the seasonal/oscillating component of the variable ###
num_forecast_steps = 6

lhs_emp_fft = fft(lhs_emp_level[:-num_forecast_steps]-
                 np.mean(lhs_emp_level[:-num_forecast_steps]))
plt.plot(np.abs(lhs_emp_fft))
plt.grid()
plt.title('LHS Emp Level FFT')
plt.show()

hsg_emp_fft = fft(hsg_emp_level[:-num_forecast_steps]-
                np.mean(hsg_emp_level[:-num_forecast_steps]))
plt.plot(np.abs(hsg_emp_fft))
plt.grid()
plt.title('HSG Emp Level FFT')
plt.show()

cad_emp_fft = fft(cad_emp_level[:-num_forecast_steps]-
                np.mean(cad_emp_level[:-num_forecast_steps]))
plt.plot(np.abs(cad_emp_fft))
plt.grid()
plt.title('CAD Emp Level FFT')
plt.show()

bhd_emp_fft = fft(bhd_emp_level[:-num_forecast_steps]-
                np.mean(bhd_emp_level[:-num_forecast_steps]))
plt.plot(np.abs(bhd_emp_fft))
plt.grid()
plt.title('BHD Emp Level FFT')
plt.show()

Once again, the above Fourier Transforms show peaks at 5 years for the LHS and HSG, at 4 years for the CAD Emp Levels, and only the usual peak at 1 year for the BHD Emp Level. We will set the number of steps per season accordingly. 

In [29]:
### To estimate the seasonal/oscillating component of the variable ###
lhs_lf_pr_fft = fft(lhs_lf_pr[:-num_forecast_steps]-
                 np.mean(lhs_lf_pr[:-num_forecast_steps]))
plt.plot(np.abs(lhs_lf_pr_fft))
plt.grid()
plt.title('LHS LF PR FFT')
plt.show()

hsg_lf_pr_fft = fft(hsg_lf_pr[:-num_forecast_steps]-
                np.mean(hsg_lf_pr[:-num_forecast_steps]))
plt.plot(np.abs(hsg_lf_pr_fft))
plt.grid()
plt.title('HSG LF PR FFT')
plt.show()

cad_lf_pr_fft = fft(cad_lf_pr[:-num_forecast_steps]-
                np.mean(cad_lf_pr[:-num_forecast_steps]))
plt.plot(np.abs(cad_lf_pr_fft))
plt.grid()
plt.title('CAD LF PR FFT')
plt.show()

bhd_lf_pr_fft = fft(bhd_lf_pr[:-num_forecast_steps]-
                np.mean(bhd_lf_pr[:-num_forecast_steps]))
plt.plot(np.abs(bhd_lf_pr_fft))
plt.grid()
plt.title('BHD LF PR FFT')
plt.show()

The above Fourier Transforms also shows the highest peak at 1 year. This suggests setting the number of steps per season to 1.

In [30]:
 ### since we have a smaller dataset for the category-wise variables ###
### LHS Labor Force Level ###
lhs_lf_model = build_linear_trend_seasonal_model(lhs_lf_level[:-num_forecast_steps],
                                                steps_per_season=2)

(lhs_lf_var_post, 
 lhs_lf_forecast_dist) = fit_model_generate_forecast(lhs_lf_model, 
                        lhs_lf_level[:-num_forecast_steps], num_forecast_steps, 
                        num_variational_steps=200)

plot_forecast_results(lhs_lf_forecast_dist, trunc_df_years, lhs_lf_level, 
                      'LHS LF Level', num_samples=10)

### HSG Labor Force Level ###
hsg_lf_model = build_linear_trend_seasonal_model(hsg_lf_level[:-num_forecast_steps],
                                                steps_per_season=2)

(hsg_lf_var_post, 
 hsg_lf_forecast_dist) = fit_model_generate_forecast(hsg_lf_model, 
                        hsg_lf_level[:-num_forecast_steps], num_forecast_steps, 
                        num_variational_steps=200)

plot_forecast_results(hsg_lf_forecast_dist, trunc_df_years, hsg_lf_level, 
                      'HSG LF Level', num_samples=10)

### CAD Labor Force Level ###
cad_lf_model = build_linear_trend_seasonal_model(cad_lf_level[:-num_forecast_steps],
                                                steps_per_season=2)
(cad_lf_var_post, 
 cad_lf_forecast_dist) = fit_model_generate_forecast(cad_lf_model, 
                        cad_lf_level[:-num_forecast_steps], num_forecast_steps, 
                        num_variational_steps=200)

plot_forecast_results(cad_lf_forecast_dist, trunc_df_years, cad_lf_level, 
                      'CAD LF Level', num_samples=10)

### BHD Labor Force Level ###
bhd_lf_model = build_linear_trend_seasonal_model(bhd_lf_level[:-num_forecast_steps])
(bhd_lf_var_post, 
 bhd_lf_forecast_dist) = fit_model_generate_forecast(bhd_lf_model, 
                        bhd_lf_level[:-num_forecast_steps], num_forecast_steps, 
                        num_variational_steps=200)

plot_forecast_results(bhd_lf_forecast_dist, trunc_df_years, bhd_lf_level, 
                      'BHD LF Level', num_samples=10)

We will now try the other more involved method, using past category-wise data, along with overall data. For this we will construct an estimate of the category-wise data using the overall data for the concerned variable and multiplying it with the population fractions for HSG and BHD, which are known.

In [31]:
lhs_lf_level_est = (1-hsg_percent_forecast/100.)*lf_level_forecast
lhs_lf_level_est = np.round(
    lhs_lf_level_est[overall_lf_level.size-lhs_lf_level.size:])

hsg_lf_level_est = ((hsg_percent_forecast - bhd_percent_forecast)/100.
                   )*lf_level_forecast
hsg_lf_level_est = np.round(
    hsg_lf_level_est[overall_lf_level.size-lhs_lf_level.size:])

bhd_lf_level_est = (bhd_percent_forecast/100.)*lf_level_forecast
bhd_lf_level_est = np.round(
    bhd_lf_level_est[overall_lf_level.size-lhs_lf_level.size:])

### for the cad level est, we will use a linear combination of 
### hsg_lf_level_est and bhd_lf_level_est. we will use the same for hsg.
cad_lf_level_est = np.transpose(np.vstack((lhs_lf_level_est, 
                                            hsg_lf_level_est, 
                                            bhd_lf_level_est)))

lhs_lf_level_est = tf.reshape(lhs_lf_level_est, (-1,1))
hsg_lf_level_est = tf.reshape(cad_lf_level_est, (-1, 3))
cad_lf_level_est = tf.reshape(cad_lf_level_est, (-1, 3))
bhd_lf_level_est = tf.reshape(bhd_lf_level_est, (-1,1))

def build_linear_seasonal_regression_model(observed_time_series, 
                                           auxilliary_time_series,
                                           steps_per_season=1):

    local_linear_trend = sts.SemiLocalLinearTrend(
                                observed_time_series=observed_time_series)    

    seasonal_trend = sts.Seasonal(
            num_seasons=2, num_steps_per_season=steps_per_season, 
            observed_time_series=observed_time_series)    
    
    regression_effect = sts.LinearRegression(
      design_matrix=auxilliary_time_series, name='regression_effect')

    model = sts.Sum([local_linear_trend, seasonal_trend, regression_effect],
                   observed_time_series=observed_time_series)
    return model

While the LHS and BHD population fractions are accurately determined by 1 - HSG and BHD population fractions, the HSG and CAD (as highest educational attainment) population fractions are not so clear. So we use both multiplied by the Overall Labor Force Level along with the BHD - HSG population fraction multiplied by the Overall Labor Force Level as regression variables for the HSG and CAD data.

In [32]:
 ### since we have a smaller dataset for the category-wise variables ###
### LHS Labor Force Level ###
lhs_lf_model_reg = build_linear_seasonal_regression_model(
                    lhs_lf_level[:-num_forecast_steps], lhs_lf_level_est, 
                    steps_per_season=2)
(lhs_lf_var_post_reg, 
 lhs_lf_forecast_dist_reg) = fit_model_generate_forecast(lhs_lf_model_reg, 
                        lhs_lf_level[:-num_forecast_steps], num_forecast_steps, 
                        num_variational_steps=200)

plot_forecast_results(lhs_lf_forecast_dist_reg, trunc_df_years, lhs_lf_level, 
                      'LHS LF Level', num_samples=10)

### HSG Labor Force Level ###
hsg_lf_model_reg = build_linear_seasonal_regression_model(
                    hsg_lf_level[:-num_forecast_steps], hsg_lf_level_est, 
                    steps_per_season=2)
(hsg_lf_var_post_reg, 
 hsg_lf_forecast_dist_reg) = fit_model_generate_forecast(hsg_lf_model_reg, 
                        hsg_lf_level[:-num_forecast_steps], num_forecast_steps, 
                        num_variational_steps=200)

plot_forecast_results(hsg_lf_forecast_dist_reg, trunc_df_years, hsg_lf_level, 
                      'HSG LF Level', num_samples=10)

### CAD Labor Force Level ###
cad_lf_model_reg = build_linear_seasonal_regression_model(
                    cad_lf_level[:-num_forecast_steps], cad_lf_level_est, 
                    steps_per_season=2)
(cad_lf_var_post_reg, 
 cad_lf_forecast_dist_reg) = fit_model_generate_forecast(cad_lf_model_reg, 
                        cad_lf_level[:-num_forecast_steps], num_forecast_steps, 
                        num_variational_steps=200)

plot_forecast_results(cad_lf_forecast_dist_reg, trunc_df_years, cad_lf_level, 
                      'CAD LF Level', num_samples=10)

### BHD Labor Force Level ###
bhd_lf_model_reg = build_linear_seasonal_regression_model(
                    bhd_lf_level[:-num_forecast_steps], bhd_lf_level_est, 
                    steps_per_season=1)
(bhd_lf_var_post_reg, 
 bhd_lf_forecast_dist_reg) = fit_model_generate_forecast(bhd_lf_model_reg, 
                        bhd_lf_level[:-num_forecast_steps], num_forecast_steps, 
                        num_variational_steps=200)

plot_forecast_results(bhd_lf_forecast_dist_reg, trunc_df_years, bhd_lf_level, 
                      'BHD LF Level', num_samples=10)

We see that while the forecasts for the BHD Labor Force Level is significantly improved, and the forecasts for the LHS and CAD Labor Force Levels is slightly improved by the regressor variables, the forecasts for the rest have either deteriorated or stayed the same. This suggests using the regressor variables for the LHS, CAD and BHD LF Level forecast, and using only the past data for the HSG LF Level forecasts.

We will now compare these two methods for the Employment Levels.

In [33]:
### since we have a smaller dataset for the category-wise variables ###
### LHS Employment Level ###
lhs_emp_model = build_linear_trend_seasonal_model(lhs_emp_level[:-num_forecast_steps],
                                                steps_per_season=2)
(lhs_emp_var_post, 
 lhs_emp_forecast_dist) = fit_model_generate_forecast(lhs_emp_model, 
                        lhs_emp_level[:-num_forecast_steps], num_forecast_steps, 
                        num_variational_steps=200)

plot_forecast_results(lhs_emp_forecast_dist, trunc_df_years, lhs_emp_level, 
                      'LHS Employment Level', num_samples=10)

### HSG Employment Level ###
hsg_emp_model = build_linear_trend_seasonal_model(hsg_emp_level[:-num_forecast_steps],
                                                steps_per_season=2)
(hsg_emp_var_post, 
 hsg_emp_forecast_dist) = fit_model_generate_forecast(hsg_emp_model, 
                        hsg_emp_level[:-num_forecast_steps], num_forecast_steps, 
                        num_variational_steps=200)

plot_forecast_results(hsg_emp_forecast_dist, trunc_df_years, hsg_emp_level, 
                      'HSG Employment Level', num_samples=10)

### CAD Employment Level ###
cad_emp_model = build_linear_trend_seasonal_model(cad_emp_level[:-num_forecast_steps],
                                                steps_per_season=2)
(cad_emp_var_post, 
 cad_emp_forecast_dist) = fit_model_generate_forecast(cad_emp_model, 
                        cad_emp_level[:-num_forecast_steps], num_forecast_steps, 
                        num_variational_steps=200)

plot_forecast_results(cad_emp_forecast_dist, trunc_df_years, cad_emp_level, 
                      'CAD Employment Level', num_samples=10)

### BHD Employment Level ###
bhd_emp_model = build_linear_trend_seasonal_model(bhd_emp_level[:-num_forecast_steps],
                                                steps_per_season=1)
(bhd_emp_var_post, 
 bhd_emp_forecast_dist) = fit_model_generate_forecast(bhd_emp_model, 
                        bhd_emp_level[:-num_forecast_steps], num_forecast_steps, 
                        num_variational_steps=200)

plot_forecast_results(bhd_emp_forecast_dist, trunc_df_years, bhd_emp_level, 
                      'BHD Employment Level', num_samples=10)

We now try using the overall data as regressors in addition to the past data trends for the employment level. 

In [34]:
lhs_emp_level_est = (1-hsg_percent_forecast/100.)*emp_level_forecast
lhs_emp_level_est = np.round(
    lhs_emp_level_est[overall_emp_level.size-lhs_emp_level.size:])

hsg_emp_level_est = ((hsg_percent_forecast - bhd_percent_forecast)/100.
                   )*emp_level_forecast
hsg_emp_level_est = np.round(
    hsg_emp_level_est[overall_emp_level.size-lhs_emp_level.size:])

bhd_emp_level_est = (bhd_percent_forecast/100.)*emp_level_forecast
bhd_emp_level_est = np.round(
    bhd_emp_level_est[overall_emp_level.size-lhs_emp_level.size:])

### for the cad level est, we will use a linear combination of 
### hsg_lf_level_est and bhd_lf_level_est. we will use the same for hsg.
cad_emp_level_est = np.transpose(np.vstack((lhs_emp_level_est, 
                                            hsg_emp_level_est, 
                                            bhd_emp_level_est)))

lhs_emp_level_est = tf.reshape(lhs_emp_level_est, (-1,1))
hsg_emp_level_est = tf.reshape(cad_emp_level_est, (-1, 3))
cad_emp_level_est = tf.reshape(cad_emp_level_est, (-1, 3))
bhd_emp_level_est = tf.reshape(bhd_emp_level_est, (-1,1))

In [35]:
 ### since we have a smaller dataset for the category-wise variables ###
### LHS Employment Level ###
lhs_emp_model_reg = build_linear_seasonal_regression_model(
                    lhs_emp_level[:-num_forecast_steps], lhs_emp_level_est, 
                    steps_per_season=2)
(lhs_emp_var_post_reg, 
 lhs_emp_forecast_dist_reg) = fit_model_generate_forecast(lhs_emp_model_reg, 
                        lhs_emp_level[:-num_forecast_steps], num_forecast_steps, 
                        num_variational_steps=200)

plot_forecast_results(lhs_emp_forecast_dist_reg, trunc_df_years, lhs_emp_level, 
                      'LHS Employment Level', num_samples=10)

### HSG Employment Level ###
hsg_emp_model_reg = build_linear_seasonal_regression_model(
                    hsg_emp_level[:-num_forecast_steps], hsg_emp_level_est, 
                    steps_per_season=2)
(hsg_emp_var_post_reg, 
 hsg_emp_forecast_dist_reg) = fit_model_generate_forecast(hsg_emp_model_reg, 
                        hsg_emp_level[:-num_forecast_steps], num_forecast_steps, 
                        num_variational_steps=200)

plot_forecast_results(hsg_emp_forecast_dist_reg, trunc_df_years, hsg_emp_level, 
                      'HSG Employment Level', num_samples=10)

### CAD Employment Level ###
cad_emp_model_reg = build_linear_seasonal_regression_model(
                    cad_emp_level[:-num_forecast_steps], cad_emp_level_est, 
                    steps_per_season=2)
(cad_emp_var_post_reg, 
 cad_emp_forecast_dist_reg) = fit_model_generate_forecast(cad_emp_model_reg, 
                        cad_emp_level[:-num_forecast_steps], num_forecast_steps, 
                        num_variational_steps=200)

plot_forecast_results(cad_emp_forecast_dist_reg, trunc_df_years, cad_emp_level, 
                      'CAD Employment Level', num_samples=10)

### BHD Employment Level ###
bhd_emp_model_reg = build_linear_seasonal_regression_model(
                    bhd_emp_level[:-num_forecast_steps], bhd_emp_level_est, 
                    steps_per_season=1)
(bhd_emp_var_post_reg, 
 bhd_emp_forecast_dist_reg) = fit_model_generate_forecast(bhd_emp_model_reg, 
                        bhd_emp_level[:-num_forecast_steps], num_forecast_steps, 
                        num_variational_steps=200)

plot_forecast_results(bhd_emp_forecast_dist_reg, trunc_df_years, bhd_emp_level, 
                      'BHD Employment Level', num_samples=10)

We again see a significant improvement in the forecasts for the BHD employment levels, a slight improvement in the variance and for the CAD employment level and in the mean for the LHS employment level, and a deterioration in the forecast for the HSG employment levels. 

This suggests using the regressor models for the LHS, CAD and BHD employment levels, and only the past data for the HSG employment level. 

We will now perform the same analysis for the category-wise Labor Force Participation Rate.

In [36]:
### since we have a smaller dataset for the category-wise variables ###
### LHS Labor Force Participation Rate ###
lhs_lf_pr_model = build_linear_trend_seasonal_model(lhs_lf_pr[:-num_forecast_steps],
                                                steps_per_season=1)
(lhs_lf_pr_var_post, 
 lhs_lf_pr_forecast_dist) = fit_model_generate_forecast(lhs_lf_pr_model, 
                        lhs_lf_pr[:-num_forecast_steps], num_forecast_steps, 
                        num_variational_steps=200)

plot_forecast_results(lhs_lf_pr_forecast_dist, trunc_df_years, lhs_lf_pr, 
                      'LHS LF Participation Rate', num_samples=10)

### HSG Labor Force Participation Rate ###
hsg_lf_pr_model = build_linear_trend_seasonal_model(hsg_lf_pr[:-num_forecast_steps],
                                                steps_per_season=1)
(hsg_lf_pr_var_post, 
 hsg_lf_pr_forecast_dist) = fit_model_generate_forecast(hsg_lf_pr_model, 
                        hsg_lf_pr[:-num_forecast_steps], num_forecast_steps, 
                        num_variational_steps=200)

plot_forecast_results(hsg_lf_pr_forecast_dist, trunc_df_years, hsg_lf_pr, 
                      'HSG LF Participation Rate', num_samples=10)

### CAD Labor Force Participation Rate ###
cad_lf_pr_model = build_linear_trend_seasonal_model(cad_lf_pr[:-num_forecast_steps],
                                                steps_per_season=1)
(cad_lf_pr_var_post, 
 cad_lf_pr_forecast_dist) = fit_model_generate_forecast(cad_lf_pr_model, 
                        cad_lf_pr[:-num_forecast_steps], num_forecast_steps, 
                        num_variational_steps=200)

plot_forecast_results(cad_lf_pr_forecast_dist, trunc_df_years, cad_lf_pr, 
                      'CAD LF Participation Rate', num_samples=10)

### BHD Labor Force Participation Rate ###
bhd_lf_pr_model = build_linear_trend_seasonal_model(bhd_lf_pr[:-num_forecast_steps],
                                                steps_per_season=1)
(bhd_lf_pr_var_post, 
 bhd_lf_pr_forecast_dist) = fit_model_generate_forecast(bhd_lf_pr_model, 
                        bhd_lf_pr[:-num_forecast_steps], num_forecast_steps, 
                        num_variational_steps=200)

plot_forecast_results(bhd_lf_pr_forecast_dist, trunc_df_years, bhd_lf_pr, 
                      'BHD LF Participation Rate', num_samples=10)

The past data appears to provide a good forecast for the future participation rates.

In [37]:
lhs_lf_pr_est = (1-hsg_percent_forecast/100.)*lf_pr_forecast
lhs_lf_pr_est = np.round(
    lhs_lf_pr_est[overall_lf_pr.size-lhs_lf_pr.size:])

hsg_lf_pr_est = ((hsg_percent_forecast - bhd_percent_forecast)/100.
                   )*lf_pr_forecast
hsg_lf_pr_est = np.round(
    hsg_lf_pr_est[overall_lf_pr.size-lhs_lf_pr.size:])

bhd_lf_pr_est = (bhd_percent_forecast/100.)*lf_pr_forecast
bhd_lf_pr_est = np.round(
    bhd_lf_pr_est[overall_lf_pr.size-lhs_lf_pr.size:])

### for the cad level est, we will use a linear combination of 
### hsg_lf_level_est and bhd_lf_level_est. we will use the same for hsg.
cad_lf_pr_est = np.transpose(np.vstack((lhs_lf_pr_est, 
                                            hsg_lf_pr_est, 
                                            bhd_lf_pr_est)))

lhs_lf_pr_est = tf.reshape(lhs_lf_pr_est, (-1,1))
hsg_lf_pr_est = tf.reshape(cad_lf_pr_est, (-1, 3))
cad_lf_pr_est = tf.reshape(cad_lf_pr_est, (-1, 3))
bhd_lf_pr_est = tf.reshape(bhd_lf_pr_est, (-1,1))

In [38]:
 ### since we have a smaller dataset for the category-wise variables ###
### LHS Labor Force Participation Rate ###
lhs_lf_pr_model_reg = build_linear_seasonal_regression_model(
                    lhs_lf_pr[:-num_forecast_steps], lhs_lf_pr_est, 
                    steps_per_season=1)
(lhs_lf_pr_var_post_reg, 
 lhs_lf_pr_forecast_dist_reg) = fit_model_generate_forecast(lhs_lf_pr_model_reg, 
                        lhs_lf_pr[:-num_forecast_steps], num_forecast_steps, 
                        num_variational_steps=200)

plot_forecast_results(lhs_lf_pr_forecast_dist_reg, trunc_df_years, lhs_lf_pr, 
                      'LHS LF Participation Rate', num_samples=10)

### HSG Labor Force Participation Rate ###
hsg_lf_pr_model_reg = build_linear_seasonal_regression_model(
                    hsg_lf_pr[:-num_forecast_steps], hsg_lf_pr_est, 
                    steps_per_season=1)
(hsg_lf_pr_var_post_reg, 
 hsg_lf_pr_forecast_dist_reg) = fit_model_generate_forecast(hsg_lf_pr_model_reg, 
                        hsg_lf_pr[:-num_forecast_steps], num_forecast_steps, 
                        num_variational_steps=200)

plot_forecast_results(hsg_lf_pr_forecast_dist_reg, trunc_df_years, hsg_lf_pr, 
                      'HSG LF Participation Rate', num_samples=10)

### CAD Labor Force Participation Rate ###
cad_lf_pr_model_reg = build_linear_seasonal_regression_model(
                    cad_lf_pr[:-num_forecast_steps], cad_lf_pr_est, 
                    steps_per_season=1)
(cad_lf_pr_var_post_reg, 
 cad_lf_pr_forecast_dist_reg) = fit_model_generate_forecast(cad_lf_pr_model_reg, 
                        cad_lf_pr[:-num_forecast_steps], num_forecast_steps, 
                        num_variational_steps=200)

plot_forecast_results(cad_lf_pr_forecast_dist_reg, trunc_df_years, cad_lf_pr, 
                      'CAD LF Participation Rate', num_samples=10)

### BHD Labor Force Participation Rate ###
bhd_lf_pr_model_reg = build_linear_seasonal_regression_model(
                    bhd_lf_pr[:-num_forecast_steps], bhd_lf_pr_est, 
                    steps_per_season=1)
(bhd_lf_pr_var_post_reg, 
 bhd_lf_pr_forecast_dist_reg) = fit_model_generate_forecast(bhd_lf_pr_model_reg, 
                        bhd_lf_pr[:-num_forecast_steps], num_forecast_steps, 
                        num_variational_steps=200)

plot_forecast_results(bhd_lf_pr_forecast_dist_reg, trunc_df_years, bhd_lf_pr, 
                      'BHD LF Participation Rate', num_samples=10)

We see that the LHS and BHD forecasts have improved slightly by using the regression variables, while the HSG and CAD forecasts have deteriorated slightly. This suggests using the regressor variables in addition to the past data for the LHS and BHD Labor Force Participation Rate forecasts, and only the past data for the HSG and CAD forecasts. 

We will now use the above defined models to generate forecasts for the years 2021-2030 for the category-wise labor force variables. Finally we will use the category-wise labor force data forecasts and the past data to generate forecasts for the category-wise median weekly incomes.

In [46]:
forecast_years = np.array(range(2021,2031))
trunc_df_forecast_yrs = np.hstack((trunc_df_years, forecast_years))
num_forecast_steps = 10

lhs_lf_level_mean, lhs_lf_level_dist = generate_forecast(lhs_lf_model_reg, 
                                                       lhs_lf_var_post_reg,
                                                       lhs_lf_level,
                                                       num_forecast_steps)
plot_forecast_future(lhs_lf_level_dist, trunc_df_forecast_yrs, lhs_lf_level, 
                     'LHS LF Level')
lhs_lf_level_forecast = np.hstack((lhs_lf_level, lhs_lf_level_mean))


hsg_lf_level_mean, hsg_lf_level_dist = generate_forecast(hsg_lf_model, 
                                                       hsg_lf_var_post,
                                                       hsg_lf_level,
                                                       num_forecast_steps)
plot_forecast_future(hsg_lf_level_dist, trunc_df_forecast_yrs, hsg_lf_level, 
                     'HSG LF Level')
hsg_lf_level_forecast = np.hstack((hsg_lf_level, hsg_lf_level_mean))


cad_lf_level_mean, cad_lf_level_dist = generate_forecast(cad_lf_model_reg, 
                                                       cad_lf_var_post_reg,
                                                       cad_lf_level,
                                                       num_forecast_steps)

plot_forecast_future(cad_lf_level_dist, trunc_df_forecast_yrs, cad_lf_level, 
                     'CAD LF Level')
cad_lf_level_forecast = np.hstack((cad_lf_level, cad_lf_level_mean))


bhd_lf_level_mean, bhd_lf_level_dist = generate_forecast(bhd_lf_model_reg, 
                                                       bhd_lf_var_post_reg,
                                                       bhd_lf_level,
                                                       num_forecast_steps)

plot_forecast_future(bhd_lf_level_dist, trunc_df_forecast_yrs, bhd_lf_level, 
                     'BHD LF Level')
bhd_lf_level_forecast = np.hstack((bhd_lf_level, bhd_lf_level_mean))

While the LHS and HSG Labor Force Levels are predicted to go down, the CAD and BHD Labor Force Levels are predicted to increase. This is in keeping with the rising percentages of high school graduation, and college attendance. 

In [47]:
lhs_emp_level_mean, lhs_emp_level_dist = generate_forecast(lhs_emp_model_reg, 
                                                       lhs_emp_var_post_reg,
                                                       lhs_emp_level,
                                                       num_forecast_steps)
plot_forecast_future(lhs_emp_level_dist, trunc_df_forecast_yrs, lhs_emp_level, 
                     'LHS Employment Level')
lhs_emp_level_forecast = np.hstack((lhs_emp_level, lhs_emp_level_mean))


hsg_emp_level_mean, hsg_emp_level_dist = generate_forecast(hsg_emp_model, 
                                                       hsg_emp_var_post,
                                                       hsg_emp_level,
                                                       num_forecast_steps)
plot_forecast_future(hsg_emp_level_dist, trunc_df_forecast_yrs, hsg_emp_level, 
                     'HSG Employment Level')
hsg_emp_level_forecast = np.hstack((hsg_emp_level, hsg_emp_level_mean))


cad_emp_level_mean, cad_emp_level_dist = generate_forecast(cad_emp_model_reg, 
                                                       cad_emp_var_post_reg,
                                                       cad_emp_level,
                                                       num_forecast_steps)
plot_forecast_future(cad_emp_level_dist, trunc_df_forecast_yrs, cad_emp_level, 
                     'CAD Employment Level')
cad_emp_level_forecast = np.hstack((cad_emp_level, cad_emp_level_mean))


bhd_emp_level_mean, bhd_emp_level_dist = generate_forecast(bhd_emp_model_reg, 
                                                       bhd_emp_var_post_reg,
                                                       bhd_emp_level,
                                                       num_forecast_steps)
plot_forecast_future(bhd_emp_level_dist, trunc_df_forecast_yrs, bhd_emp_level, 
                     'BHD Employment Level')
bhd_emp_level_forecast = np.hstack((bhd_emp_level, bhd_emp_level_mean))

Due the recent setback of the pandemic, most employment levels are predicted to either stagnate or decrease and then stagnate.

In [48]:
lhs_lf_pr_mean, lhs_lf_pr_dist = generate_forecast(lhs_lf_pr_model_reg, 
                                                       lhs_lf_pr_var_post_reg,
                                                       lhs_lf_pr,
                                                       num_forecast_steps)
plot_forecast_future(lhs_lf_pr_dist, trunc_df_forecast_yrs, lhs_lf_pr, 
                     'LHS LF Participation Rate')
lhs_lf_pr_forecast = np.hstack((lhs_lf_pr, lhs_lf_pr_mean))


hsg_lf_pr_mean, hsg_lf_pr_dist = generate_forecast(hsg_lf_pr_model, 
                                                       hsg_lf_pr_var_post,
                                                       hsg_lf_pr,
                                                       num_forecast_steps)
plot_forecast_future(hsg_lf_pr_dist, trunc_df_forecast_yrs, hsg_lf_pr, 
                     'HSG LF Participation Rate')
hsg_lf_pr_forecast = np.hstack((hsg_lf_pr, hsg_lf_pr_mean))


cad_lf_pr_mean, cad_lf_pr_dist = generate_forecast(cad_lf_pr_model, 
                                                       cad_lf_pr_var_post,
                                                       cad_lf_pr,
                                                       num_forecast_steps)
plot_forecast_future(cad_lf_pr_dist, trunc_df_forecast_yrs, cad_lf_pr, 
                     'CAD LF Participation Rate')
cad_lf_pr_forecast = np.hstack((cad_lf_pr, cad_lf_pr_mean))


bhd_lf_pr_mean, bhd_lf_pr_dist = generate_forecast(bhd_lf_pr_model_reg, 
                                                       bhd_lf_pr_var_post_reg,
                                                       bhd_lf_pr,
                                                       num_forecast_steps)
plot_forecast_future(bhd_lf_pr_dist, trunc_df_forecast_yrs, bhd_lf_pr, 
                     'BHD LF Participation Rate')
bhd_lf_pr_forecast = np.hstack((bhd_lf_pr, bhd_lf_pr_mean))

Except for the LHS category, the Labor Force participation rates for all other categories have been decreasing steadily, without a significant change due to the pandemic, and the trend is predicted to continue.

We will now estimate the median weekly earnings (MWE) for each educational attainment category by two methods: (i) using all the historical median weekly earnings data (1979 onwards) for that category, and (ii) using the historical MWE data for that category (1992 onwards) and the labor force data for that category as regression variables, since the category-wise labor force data is available only for years 1992 onwards.

In [52]:
lhs_mwe_full = (full_df['LHS MEDIAN WEEKLY EARNING (IN 2020 DOLLARS)'].dropna()
               ).to_numpy()
hsg_mwe_full = (full_df['HSG MEDIAN WEEKLY EARNING (IN 2020 DOLLARS)'].dropna()
               ).to_numpy()
cad_mwe_full = (full_df['CAD MEDIAN WEEKLY EARNING (IN 2020 DOLLARS)'].dropna()
               ).to_numpy()
bhd_mwe_full = (full_df['BHD MEDIAN WEEKLY EARNING (IN 2020 DOLLARS)'].dropna()
               ).to_numpy()
full_mwe_years = (full_df['YEAR'].to_numpy())[-lhs_mwe_full.size:]


lhs_mwe_trunc = (truncated_df['LHS MEDIAN WEEKLY EARNING (IN 2020 DOLLARS)'].dropna()
               ).to_numpy()
hsg_mwe_trunc = (truncated_df['HSG MEDIAN WEEKLY EARNING (IN 2020 DOLLARS)'].dropna()
               ).to_numpy()
cad_mwe_trunc = (truncated_df['CAD MEDIAN WEEKLY EARNING (IN 2020 DOLLARS)'].dropna()
               ).to_numpy()
bhd_mwe_trunc = (truncated_df['BHD MEDIAN WEEKLY EARNING (IN 2020 DOLLARS)'].dropna()
               ).to_numpy()

(i) Using only the historical MWE data (1979 onwards):

In [53]:
num_forecast_steps = 6
### To estimate the seasonal/oscillating component of the MWE ###
lhs_mwe_full_fft = fft(lhs_mwe_full[:-num_forecast_steps]-
                 np.mean(lhs_mwe_full[:-num_forecast_steps]))
plt.plot(np.abs(lhs_mwe_full_fft))
plt.grid()
plt.title('LHS MWE FFT')
plt.show()

hsg_mwe_full_fft = fft(hsg_mwe_full[:-num_forecast_steps]-
                np.mean(hsg_mwe_full[:-num_forecast_steps]))
plt.plot(np.abs(hsg_mwe_full_fft))
plt.grid()
plt.title('HSG MWE FFT')
plt.show()

cad_mwe_full_fft = fft(cad_mwe_full[:-num_forecast_steps]-
                np.mean(cad_mwe_full[:-num_forecast_steps]))
plt.plot(np.abs(cad_mwe_full_fft))
plt.grid()
plt.title('CAD MWE FFT')
plt.show()

bhd_mwe_full_fft = fft(bhd_mwe_full[:-num_forecast_steps]-
                np.mean(bhd_mwe_full[:-num_forecast_steps]))
plt.plot(np.abs(bhd_mwe_full_fft))
plt.grid()
plt.title('BHD MWE FFT')
plt.show()

We see that while the LHS and BHD MWE data have FFT peaks at 1 year, the HSG and CAD MWE data have secondary peaks at 4 years.

In [54]:
### since we have a smaller dataset for the category-wise variables ###
### LHS MWE ###
lhs_mwe_model = build_linear_trend_seasonal_model(lhs_mwe_full[:-num_forecast_steps])
(lhs_mwe_var_post, 
 lhs_mwe_forecast_dist) = fit_model_generate_forecast(lhs_mwe_model, 
                        lhs_mwe_full[:-num_forecast_steps], num_forecast_steps, 
                        num_variational_steps=200)

plot_forecast_results(lhs_mwe_forecast_dist, full_mwe_years, lhs_mwe_full, 
                      'LHS MWE', num_samples=10)

### HSG MWE ###
hsg_mwe_model = build_linear_trend_seasonal_model(hsg_mwe_full[:-num_forecast_steps],
                                                steps_per_season=2)
(hsg_mwe_var_post, 
 hsg_mwe_forecast_dist) = fit_model_generate_forecast(hsg_mwe_model, 
                        hsg_mwe_full[:-num_forecast_steps], num_forecast_steps, 
                        num_variational_steps=200)

plot_forecast_results(hsg_mwe_forecast_dist, full_mwe_years, hsg_mwe_full, 
                      'HSG MWE', num_samples=10)

### CAD MWE ###
cad_mwe_model = build_linear_trend_seasonal_model(cad_mwe_full[:-num_forecast_steps],
                                                steps_per_season=2)
(cad_mwe_var_post, 
 cad_mwe_forecast_dist) = fit_model_generate_forecast(cad_mwe_model, 
                        cad_mwe_full[:-num_forecast_steps], num_forecast_steps, 
                        num_variational_steps=200)

plot_forecast_results(cad_mwe_forecast_dist, full_mwe_years, cad_mwe_full, 
                      'CAD MWE', num_samples=10)

### BHD MWE ###
bhd_mwe_model = build_linear_trend_seasonal_model(bhd_mwe_full[:-num_forecast_steps],
                                                steps_per_season=1)
(bhd_mwe_var_post, 
 bhd_mwe_forecast_dist) = fit_model_generate_forecast(bhd_mwe_model, 
                        bhd_mwe_full[:-num_forecast_steps], num_forecast_steps, 
                        num_variational_steps=200)

plot_forecast_results(bhd_mwe_forecast_dist, full_mwe_years, bhd_mwe_full, 
                      'BHD MWE', num_samples=10)

We see that the forecasts based only on historical trends show a downward trend for all but the BHD category.

We will now use the labor force variables as regressors in addition to historical data for predicting the category-wise median weekly earnings.

In [55]:
#### normalize the labor force variables #### to make them within 0-10
lf_level_normalization = 1./10000.
emp_level_normalization = 1./10000.
lf_pr_normalization = 1./10.

lhs_lf_level_reg = lhs_lf_level_forecast*lf_level_normalization
hsg_lf_level_reg = hsg_lf_level_forecast*lf_level_normalization
cad_lf_level_reg = cad_lf_level_forecast*lf_level_normalization
bhd_lf_level_reg = bhd_lf_level_forecast*lf_level_normalization

lhs_emp_level_reg = lhs_emp_level_forecast*emp_level_normalization
hsg_emp_level_reg = hsg_emp_level_forecast*emp_level_normalization
cad_emp_level_reg = cad_emp_level_forecast*emp_level_normalization
bhd_emp_level_reg = bhd_emp_level_forecast*emp_level_normalization

lhs_lf_pr_reg = lhs_lf_pr_forecast*lf_pr_normalization
hsg_lf_pr_reg = hsg_lf_pr_forecast*lf_pr_normalization
cad_lf_pr_reg = cad_lf_pr_forecast*lf_pr_normalization
bhd_lf_pr_reg = bhd_lf_pr_forecast*lf_pr_normalization

lhs_mwe_est = np.transpose(np.vstack((lhs_lf_level_reg, 
                                    lhs_emp_level_reg, 
                                    lhs_lf_pr_reg)))
hsg_mwe_est = np.transpose(np.vstack((hsg_lf_level_reg, 
                                    hsg_emp_level_reg, 
                                    hsg_lf_pr_reg)))
cad_mwe_est = np.transpose(np.vstack((cad_lf_level_reg, 
                                    cad_emp_level_reg, 
                                    cad_lf_pr_reg)))
bhd_mwe_est = np.transpose(np.vstack((bhd_lf_level_reg, 
                                    bhd_emp_level_reg, 
                                    bhd_lf_pr_reg)))

lhs_mwe_est = tf.reshape(lhs_mwe_est, (-1,3))
hsg_mwe_est = tf.reshape(cad_mwe_est, (-1,3))
cad_mwe_est = tf.reshape(cad_mwe_est, (-1,3))
bhd_mwe_est = tf.reshape(bhd_mwe_est, (-1,3))

In [56]:
lhs_mwe_model_reg = build_linear_seasonal_regression_model(
                            lhs_mwe_trunc[:-num_forecast_steps],
                            lhs_mwe_est, steps_per_season=1)
(lhs_mwe_var_post_reg, 
 lhs_mwe_forecast_dist_reg) = fit_model_generate_forecast(lhs_mwe_model_reg, 
                        lhs_mwe_trunc[:-num_forecast_steps], num_forecast_steps, 
                        num_variational_steps=200)
plot_forecast_results(lhs_mwe_forecast_dist_reg, trunc_df_years, lhs_mwe_trunc, 
                      'LHS MWE', num_samples=10)


hsg_mwe_model_reg = build_linear_seasonal_regression_model(
                            hsg_mwe_trunc[:-num_forecast_steps],
                            hsg_mwe_est, steps_per_season=2)
(hsg_mwe_var_post_reg, 
 hsg_mwe_forecast_dist_reg) = fit_model_generate_forecast(hsg_mwe_model_reg, 
                        hsg_mwe_trunc[:-num_forecast_steps], num_forecast_steps, 
                        num_variational_steps=200)
plot_forecast_results(hsg_mwe_forecast_dist_reg, trunc_df_years, hsg_mwe_trunc, 
                      'HSG MWE', num_samples=10)


cad_mwe_model_reg = build_linear_seasonal_regression_model(
                            cad_mwe_trunc[:-num_forecast_steps],
                            cad_mwe_est, steps_per_season=2)
(cad_mwe_var_post_reg, 
 cad_mwe_forecast_dist_reg) = fit_model_generate_forecast(cad_mwe_model_reg, 
                        cad_mwe_trunc[:-num_forecast_steps], num_forecast_steps, 
                        num_variational_steps=200)
plot_forecast_results(cad_mwe_forecast_dist_reg, trunc_df_years, cad_mwe_trunc, 
                      'CAD MWE', num_samples=10)


bhd_mwe_model_reg = build_linear_seasonal_regression_model(
                            bhd_mwe_trunc[:-num_forecast_steps],
                            bhd_mwe_est, steps_per_season=1)
(bhd_mwe_var_post_reg, 
 bhd_mwe_forecast_dist_reg) = fit_model_generate_forecast(bhd_mwe_model_reg, 
                        bhd_mwe_trunc[:-num_forecast_steps], num_forecast_steps, 
                        num_variational_steps=200)
plot_forecast_results(bhd_mwe_forecast_dist_reg, trunc_df_years, bhd_mwe_trunc, 
                      'BHD MWE', num_samples=10)

We see that using the regressor variables mitigates the overall downward trend for most of the forecasts of the median weekly earnings. While using the regressor variables does not lead to a significant improvement in performance, it does not cause a significant deterioration either, and leads to a better trend overall. 

We will generate forecasts for the category-wise median weekly earnings using both methods (i) only the historical median weekly earnings data, and (ii) the labor force data forecasts for each category in addition to the historical median weekly earnings data.

In [61]:
#### METHOD (I) ####
forecast_years = np.array(range(2021,2031))
mwe_forecast_yrs = np.hstack((full_mwe_years, forecast_years))
num_forecast_steps = 10

lhs_mwe_mean, lhs_mwe_dist = generate_forecast(lhs_mwe_model, 
                                                       lhs_mwe_var_post,
                                                       lhs_mwe_full,
                                                       num_forecast_steps)
plot_forecast_future(lhs_mwe_dist, mwe_forecast_yrs, lhs_mwe_full, 
                     'LHS Median Weekly Earnings')


hsg_mwe_mean, hsg_mwe_dist = generate_forecast(hsg_mwe_model, 
                                                       hsg_mwe_var_post,
                                                       hsg_mwe_full,
                                                       num_forecast_steps)
plot_forecast_future(hsg_mwe_dist, mwe_forecast_yrs, hsg_mwe_full, 
                     'HSG Median Weekly Earnings')


cad_mwe_mean, cad_mwe_dist = generate_forecast(cad_mwe_model, 
                                                       cad_mwe_var_post,
                                                       cad_mwe_full,
                                                       num_forecast_steps)
plot_forecast_future(cad_mwe_dist, mwe_forecast_yrs, cad_mwe_full, 
                     'CAD Median Weekly Earnings')


bhd_mwe_mean, bhd_mwe_dist = generate_forecast(bhd_mwe_model, 
                                                       bhd_mwe_var_post,
                                                       bhd_mwe_full,
                                                       num_forecast_steps)
plot_forecast_future(bhd_mwe_dist, mwe_forecast_yrs, bhd_mwe_full, 
                     'BHD Median Weekly Earnings')

We see that in keeping with the earlier forecast, using only the historical data suggests a further downward trend for the median weekly earnings for each category except for BHD -- persons with Bachelors or Higher Degree. 

We will now generate a forecast using both the historical data and forecasts for the labor force data.

In [60]:
#### METHOD (II) ####
forecast_years = np.array(range(2021,2031))
mwe_forecast_yrs = np.hstack((full_mwe_years, forecast_years))
num_forecast_steps = 10

lhs_mwe_mean, lhs_mwe_dist = generate_forecast(lhs_mwe_model_reg, 
                                                       lhs_mwe_var_post_reg,
                                                       lhs_mwe_trunc,
                                                       num_forecast_steps)
plot_forecast_future(lhs_mwe_dist, mwe_forecast_yrs, lhs_mwe_full, 
                     'LHS Median Weekly Earnings')


hsg_mwe_mean, hsg_mwe_dist = generate_forecast(hsg_mwe_model_reg, 
                                                       hsg_mwe_var_post_reg,
                                                       hsg_mwe_trunc,
                                                       num_forecast_steps)
plot_forecast_future(hsg_mwe_dist, mwe_forecast_yrs, hsg_mwe_full, 
                     'HSG Median Weekly Earnings')


cad_mwe_mean, cad_mwe_dist = generate_forecast(cad_mwe_model_reg, 
                                                       cad_mwe_var_post_reg,
                                                       cad_mwe_trunc,
                                                       num_forecast_steps)
plot_forecast_future(cad_mwe_dist, mwe_forecast_yrs, cad_mwe_full, 
                     'CAD Median Weekly Earnings')


bhd_mwe_mean, bhd_mwe_dist = generate_forecast(bhd_mwe_model_reg, 
                                                       bhd_mwe_var_post_reg,
                                                       bhd_mwe_trunc,
                                                       num_forecast_steps)
plot_forecast_future(bhd_mwe_dist, mwe_forecast_yrs, bhd_mwe_full, 
                     'BHD Median Weekly Earnings')

Using the Labor force data forecasts, we see an upward trend for the median weekly earnings for all categories except LHS (persons with Less than a High School graduation). However, the HSG and CAD categories show a stagnation of the median weekly earnings after a few years, and it is only the BHD category which shows a continous upward trend for the next decade. 

The conclusion is that for persons without a bachelors degree, earnings are going to stagnate or decrease in the coming years. This suggests that the government must try to make college education accessible to all.